# TP 7 - SVM linéaire et validation croisée

**L’objectif** de cette séance de travaux pratiques est de montrer l’utilisation des techniques de validation croisée pour l’évaluation et la comparaison de modèles décisionnels, ainsi que des méthodes de recherche de valeurs pour les hyper-paramètres (comme le coefficient de régularisation).

## 1 - Estimation des performances par validation croisée

Afin d’illustrer l’utilisation de la validation croisée, nous considérons un problème de classification sur le dataset Iris. Pour rappel, celui-ci consiste à identifier à quelle espèce appartient une plante parmi trois possibilités (*Iris Setosa*, *Iris Virginica* et *Iris Versicolor*) à partir de la longueur et la largeur des sépales et des pétales.

Nous employons des perceptrons multi-couches (PMC) avec une seule couche cachée de 100 neurones et une valeur $ \alpha = 1 $ pour la constante de régularisation (pondération du terme d’oubli ou *weight decay*). Il n’est pas indispensable pour cette séance de savoir en détails comment fonctionnent ces réseaux de neurones.

La validation croisée sera utilisée pour estimer les performances de généralisation *à partir de l’ensemble d’apprentissage*. Ensuite, cette estimation sera comparée à l’estimation obtenue sur l’ensemble de test que nous avions mis de côté au départ. Les explications sur la validation croisée et sa mise en œuvre dans Scikit-learn se trouvent [dans la documentation](http://scikit-learn.org/stable/modules/cross_validation.html).

In [ ]:
# importations
import numpy as np
import matplotlib.pyplot as plt
import sklearn

# chargement du jeu de données Iris
from sklearn import datasets
iris = datasets.load_iris()
data, labels = iris.data, iris.target

# découpage initial en données d'apprentissage et données de test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.5)

La fonction train_test_split de scikit-learn nous permet de diviser aléatoirement le jeu de données en deux partitions train (apprentissage) et test (évaluation) selon des proportions arbitraires.

Comme d’habitude, il est intéressant de visualiser les données à notre disposition. Nous pouvons construire les nuages de points en deux dimensions à l’aide de Matplotlib :

In [ ]:
fig = plt.figure(figsize=(16, 12))
n_features = data.shape[-1]
n_plots = 6
idx = 1
cmp = np.array(['r', 'g', 'b'])
for dim1 in range(0, n_features):
    for dim2 in range(dim1+1, n_features):
        fig.add_subplot(2, n_plots // 2, idx)
        plt.scatter(X_train[:, dim1], X_train[:, dim2],c=cmp[y_train], s=50, edgecolors='none')
        plt.scatter(X_test[:,  dim1], X_test[:, dim2], c='none',  s=50, edgecolors=cmp[y_test])
        plt.xlabel(iris.feature_names[dim1])
        plt.ylabel(iris.feature_names[dim2])
        idx += 1
plt.show()

## Question

Pourquoi a-t-on six nuages de points différents ? Que représente chaque figure ?

...

Pour ce problème de classement, nous allons utiliser un perceptron multi-couches (MLP) avec les valeurs par défaut proposés par scikit-learn pour le nombre de couches (1) et de neurones (100).

In [ ]:
# emploi de PMC
from sklearn.neural_network import MLPClassifier
clf = MLPClassifier(solver='lbfgs', alpha=1, tol=5e-3)

Pour estimer l’erreur de généralisation, nous allons utiliser la validation croisée *K-fold*. Cela va nous permettre d’ajuster, si besoin, les hyperparamètres du modèle décisionnel. scikit-learn implémente diverses stratégies de validation croisée dans le module sklearn.model_selection. Commençons par expérimenter avec l’approche *K-fold*. L’objet KFold dispose d’une méthode .split() qui permet de générer les listes des indices des observations à utiliser pour le sous-jeu d’apprentissage et pour le sous-jeu de *validation*. Plus de détails sur cet objet se trouvent dans la [documentation de K-Fold](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.KFold.html).

In [ ]:
# KFold pour différentes valeurs de k
from sklearn.model_selection import KFold

# valeurs de k
n_folds = np.array([2, 3, 5, 7, 10, 13, 16])

# préparation des listes pour stocker les résultats
cv_scores = []
cv_scores_std = []

for k in n_folds:    # pour chaque valeur de k
    kf = KFold(n_splits=k)
    scores = []
    # apprentissage puis évaluation d'un modèle sur chaque split
    for train_idx, val_idx in kf.split(X_train):
        # apprentissage avec .fit()
        clf.fit(X_train[train_idx], y_train[train_idx])
        scores.append(clf.score(X_train[val_idx], y_train[val_idx]))
    # calcul de la moyenne et de l'écart-type des performances obtenues
    cv_scores.append(np.mean(scores))
    cv_scores_std.append(np.std(scores))

cv_scores, cv_scores_std = np.array(cv_scores), np.array(cv_scores_std)

# affichage performance moyenne +- 1 écart-type pour chaque k
plt.figure(figsize=(8, 6))
plt.plot(n_folds, cv_scores, 'b')
plt.fill_between(n_folds, cv_scores + cv_scores_std, cv_scores - cv_scores_std, alpha=0.5)
plt.xlabel("Valeur de $k$ du K-Fold")
plt.ylabel("Score moyen")
plt.xlim(2, max(n_folds))
plt.xticks(n_folds)
plt.title("Erreur de généralisation estimée en fonction de $k$")
plt.show()

## Question :

Que constatez-vous en examinant ce graphique ? Ajoutez des valeurs pour *k* (par ex. 20, 40...) et affichez de nouveau le graphique.

## Question :

Pour chaque modèle appris par validation croisée *k-fold*, ajoutez son évaluation sur les données de test mises de côté au départ `X_test, y_test`. Affichez les courbes sur le même graphique. Que constatez-vous ?

## Question :

Réalisez l’estimation des performances en utilisant la [validation croisée leave one out (LOO)](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.LeaveOneOut.html#sklearn.model_selection.LeaveOneOut). Que constatez-vous en comparant les résultats de *k-fold* et de *leave one out* ?

In [ ]:
from sklearn.model_selection import LeaveOneOut

loo = LeaveOneOut()


scores = []
test_scores = []
# apprentissage puis évaluation d'un modèle sur chaque split
for train_idx, val_idx in loo.split(X_train):
    ...
# calcul de la moyenne et de l'écart-type des performances obtenues
mean_score = ...
std_score = ...
test_score = ...

print('loo_score =', mean_score
print('loo_std =', std_score)
print('test_score =', test_score)

## Note

Pour l’illustration, nous avons itéré ici manuellement à l’aide des itérateurs KFold et LeaveOneOut. Toutefois il est possible d’automatiser ce procédé dans le cas du *KFold* à l’aide de la fonction cross_val_score:

In [ ]:
from sklearn.model_selection import cross_val_score
clf = MLPClassifier(solver='lbfgs', alpha=1, tol=5e-3)

# Cross-validation KFold avec k=5
scores = cross_val_score(clf, X_train, y_train, cv=5)

print('Scores de validation avec k=5 : ', scores)
print('Score moyen : ', np.mean(scores))


Cette fonction prend des paramètres supplémentaires (métriques, parallélisation, etc.) qui sont décrites dans [sa documentation](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_score.html#sklearn.model_selection.cross_val_score).


<a id='chap-tpsvmlineaires'></a>

# 2 - SVM linéaires


L’objectif est d’illustration l’utilisation des machines à vecteurs de support  (*Support Vector Machines*, SVM) pour les problèmes de classification linéaires en python avec Scikit-learn.

Références externes utiles :

> - [Documentation NumPy](https://docs.scipy.org/doc/numpy/user/index.html)  
- [Documentation SciPy](https://docs.scipy.org/doc/scipy/reference/)  
- [Documentation MatPlotLib](http://matplotlib.org/)  
- [Site scikit-learn](http://scikit-learn.org/stable/index.html)  
- [Site langage python](https://www.python.org)  
- [Site LibSVM](https://www.csie.ntu.edu.tw/~cjlin/libsvm/)  
- [Site LibLinear](https://www.csie.ntu.edu.tw/~cjlin/liblinear/)  

## Machines à vecteurs de support linéaires

Les machines à vecteurs de support (SVM : *Support Vector Machines*) sont une classe de méthodes d’apprentissage statistique basées sur le principe de la maximisation de la marge (séparation des classes). Il existe plusieurs formulations (linéaires, versions à noyaux) qui peuvent s’appliquer sur des données séparables (linéairement) mais aussi sur des données non séparables.

Les avantages des SVM :

- Très efficaces en dimension élevée.  
- Ils sont aussi efficaces dans le cas où la dimension de l’espace est plus grande que le nombre d’échantillons d’apprentissage.  
- Pour la décision, n’utilisent pas tous les échantillons d’apprentissage, mais seulement une partie (les vecteurs de support). En conséquence, ces algorithmes demandent moins de mémoire.  


Désavantages :

- Si le nombre d’attributs est beaucoup plus grand que le nombre d’échantillons, les performances sont moins bonnes.  
- Comme il s’agit de méthodes de discrimination entre les classes, elles ne fournissent pas d’estimations de probabilités.  

On continue avec le dataset iris. Pour commencer, nous ne conservons que les deux premiers attributs du jeu de données :

In [ ]:
X, y = iris.data[:, :2], iris.target
# On conserve 50% du jeu de données pour l'évaluation
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5)

Nous pouvons maintenant entraîner une machine à vecteur de support linéaire :

In [ ]:
from sklearn import svm

C = 1.0 # paramètre de régularisation
lin_svc = svm.LinearSVC(C=C)
lin_svc.fit(X_train, y_train)

## Question

Calculez le score d’échantillons bien classifiés sur le jeu de données de test.

Visualisons maintenant la surface de décision apprise par notre modèle :

In [ ]:
# Créer la surface de décision discretisée
x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
# Pour afficher la surface de décision on va discrétiser l'espace avec un pas h
h = max((x_max - x_min) / 100, (y_max - y_min) / 100)
xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))

# Surface de décision
Z = lin_svc.predict(np.c_[xx.ravel(), yy.ravel()])
Z = Z.reshape(xx.shape)

plt.contourf(xx, yy, Z, cmap=plt.cm.coolwarm, alpha=0.8)
# Afficher aussi les points d'apprentissage
plt.scatter(X_train[:, 0], X_train[:, 1], label="train", edgecolors='k', c=y_train, cmap=plt.cm.coolwarm)
plt.scatter(X_test[:, 0], X_test[:, 1], label="test", marker='*', c=y_test, cmap=plt.cm.coolwarm)
plt.xlabel('Sepal length')
plt.ylabel('Sepal width')
plt.title("LinearSVC")

## Question :

Afficher de nouveau les frontières de décision en testant différentes valeurs pour le paramètre C. Comment la frontière de décision évolue en fonction de C ?

## Question

On va maintenant utiliser la validation croisée pour tester différentes valeurs du paramètre `C`. La fonction `GridSearchCV` de scikit-learn va s'occuper de tester les différentes valeurs pour nous. Afficher les scores de validation obtenus pour les différentes valeurs de `C`.

In [ ]:
from sklearn.model_selection import GridSearchCV
parameters = {'C':[1,3,5,10,20,50,100,1000]}

...

## Question

D’après la visualisation des frontières de décision, ce modèle vous paraît-il adapté au problème ? Si non, que peut-on faire pour l’améliorer ?

...

Pour l’instant, nous n’avons exploité que deux variables explicatives. Néanmoins, l’intérêt des machines à vecteur de support linéaires est qu’il est souvent plus facile de trouver des hyperplans séparateurs dans des espaces de grande dimension.

## Question

Réalisez l’optimisation d’une nouvelle machine à vecteur de support linéaire mais en utilisant les quatre attributs du jeu de données Iris. Le score de classification en test a-t-il augmenté ? Pourquoi ?

In [ ]:
C = 1

...

...

## 3 - Jeu de données Digits

Le jeu de données Digits est une collection d’images de chiffres manuscrits. Elles peuvent se charger directement depuis scikit-learn :

In [ ]:
from sklearn.datasets import load_digits
digits = load_digits()
X, y = digits.data, digits.target

## Question :

Utilisez les données Digits pour construire un classifieur LinearSVC et évaluez-le. Si le temps d’apprentissage est trop long, sélectionnez une partie plus petite de la base d’apprentissage (par exemple 10000 échantillons). Pour quelle valeur de C on obtient le meilleurs résultats de généralisation ? Utiliser la validation croisée avec `GridSearchCV`.